<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD Tutorial

FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the [CeRAS reference case](http://ceras.ilr.rwth-aachen.de/trac/wiki/CeRAS/AircraftDesigns/CSR01). In `workdir/`, we store files generated or modified by FAST-OAD.

In [ ]:
import os.path as pth
import openmdao.api as om
from fastoad import api
import logging
from fastoad.utils.postprocessing import VariableViewer
import shutil

DATA_FOLDER_PATH = 'data'

WORK_FOLDER_PATH = 'workdir'

CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'oad_process.yml')
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, 'CeRAS01_baseline.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

After defining a configuration file name, we can ask FAST-OAD to generate a default configuration file based on the default OAD model implemented in the framework:

In [ ]:
api.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

# The above generated configuration uses the quick and simple Breguet module to compute performances. 
# If you want to use the more accurate, much more CPU-costly time-step integration, uncomment and run 
# the next line, but don't do the optimization part (will be fixed after this alpha release):
#shutil.copy(pth.join(DATA_FOLDER_PATH, "oad_process_timestep_mission.yml"), CONFIGURATION_FILE)

You can now checkout the generated [configuration file](./workdir/oad_process.yml).

In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [ ]:
api.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [ ]:
api.list_variables(CONFIGURATION_FILE)

Another useful feature is to list the modules of the model defined in the configuration file:

In [ ]:
api.list_systems(CONFIGURATION_FILE)

Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

Alternatively, you can create a [WhatsOpt](https://github.com/OneraHub/WhatsOpt-Doc#whatsopt-documentation) account to generate the XDSM of the problem. If your account is created, you may uncomment next lines and run them (this should take ~ 1 min):

In [ ]:
# XDSM_FILE = pth.join(WORK_FOLDER_PATH, 'xdsm.html')
# api.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)
# from IPython.display import IFrame
# IFrame(src=XDSM_FILE, width='100%', height='500px')

## 2. Running your first MDA

### CeRAS
Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

In [ ]:
eval_problem = api.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

Let's save these results. We will use them in the next workbook, that shows some post-processing utilities.

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_CeRAS_2750nm_mda.xml')
shutil.copy(OUTPUT_FILE, CeRAS_OUTPUT_FILE)

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [ ]:
api.variable_viewer(OUTPUT_FILE)

In particular, you may inspect the `data:handling_qualities:static_margin` variable.
You will see that its value is slightly negative, which means that the current aircraft configuration is estimated as unstable. We will see later how we can correct this point.

### CeRAS for 800 nm range
Here we run an MDA but we change one of the Top Level Aircraft Requirement (TLAR): the range. We choose a 800 nm range instead of the 2400 nm of the CeRAS. For that we use the `VariableViewer` tool on the input file to change the range (do not forget to save!). Just like this:
![variable_viewer](./img/variable_viewer_change_range.gif)

In [ ]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs.xml')
api.variable_viewer(INPUT_FILE)

Now that the range has been changed, we run again the MDA.

In [ ]:
eval_problem = api.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

Let's save again these new results, for post-processing them in next notebook.

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_800nm_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_CeRAS_800nm_mda.xml')
shutil.copy(OUTPUT_FILE, CeRAS_800nm_OUTPUT_FILE)

## 3. Running your first MDO

## CeRAS 2750 nm optimization

As seen earlier, the current aircraft configuration is estimated as unstable. We will fix this using a simple optimization problem that will move wing position so the static margin is at least +5%, while keeping the fuel consumption as low as possible. 

The default configuration file defines this optimization problem that aims at:
- minimizing the fuel consumption for the mission (objective),
- with respect to the wing position (design variables),
- subject to a static margin (constraints).

*(This run should take a few dozen seconds)*

In [ ]:
# Set back the inputs from the reference CeRAS 2750 nm
api.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

To visualize and edit the optimization problem definition (present in the configuration file .yml) you can use the `optimization_viewer` tool. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated. Modifiying the `Initial Value` will modify the input file defined in the configuration file .yml whereas `Value` corresponds to the value found in the output file defined in the configuration file (here it is the 800 nm MDA run).

In [ ]:
api.optimization_viewer(CONFIGURATION_FILE)

In [ ]:
optim_problem = api.optimize_problem(CONFIGURATION_FILE, overwrite=True)

Let's save these results:

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_OPT_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_CeRAS_2750nm_mdo.xml')
shutil.copy(OUTPUT_FILE, CeRAS_OPT_OUTPUT_FILE)

The `optimizer_viewer` offers a convenient summary of the optimization result:

In [ ]:
api.optimization_viewer(CONFIGURATION_FILE)

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [ ]:
RESULT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')

In [ ]:
api.variable_viewer(RESULT_FILE)